### Association rules by Babies
#### Department - Babies

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [3]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [4]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [5]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [6]:
combine_train_prior

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
2,2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,3,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
4,4,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,fresh fruits,produce
5,5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,6,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
7,7,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
8,8,36,46979,8,1,79431,train,23,6,18,30.0,Asparagus,83,4,fresh vegetables,produce
9,9,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="babies"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [9]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=3)

In [10]:
grouped_data

,order_id,product_name
18,7755,Organic Pears & Rasberries Stage 2 Baby Food
19,7755,Organic Baby Foods Whole Grain Multi-Grain Cereal
20,7755,Organic Infant Formula With Iron
22,8121,Stage 2 Chicken Casserole with Vegetables & Ri...
23,8121,Organic Fiber & Protein Pear Blueberry & Spina...
24,8121,"Fiber & Protein Organic Pears, Raspberries, Bu..."
25,8121,Baby Food Meals
26,8121,Organic Stage 4 Green Bean Pear & Pea Baby Food
27,8121,"Super Foods/Organic Apples, Spinach, Peas & Br..."
35,12768,Organic Go Go Apple Peach Applesauce


In [11]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [12]:
table1

product_name,1F Organic Apples Organic Purees Fruit,1F Organic Bananas Organic Purees Fruit,1F Organic Mixed Carrots Organic Purees Vegetable,1F Organic Oatmeal Cereal Organic Cereal,1pk 270ct Refill,1st Foods Applesauce Baby Food,1st Foods Baby Food- Pears,1st Foods Bananas Baby Food,1st Foods Green Beans,1st Foods Peaches,...,Yellow Rice & Chicken with Vegetables in Sauce with Green Beans & Carrots,YoTot Apple Sweet Potato with Fruit & Veggies Organic Whole Milk Shutterfly Promo Organic Yogurt,Yogurt Blends Apple Cinnamon,Yogurt Blends Sweet Potato Pear,"Yogurt, Banana and Mixed Berry Blend Breakfast On-The-Go","Yum-A-Roo's Organic Toddler Snacks Pea, Banana, & Pineapple","Yum-a-roo's Organic Toddler Snacks Banana, Mango & Pineapple",Zucchini Banana & Amaranth Organic Baby Food,"Zucchini, Spinach & Banana Blend Veggies On-The-Go Stage 2 (6 Months and Up)",in Gravy with Carrots Peas & Corn Mashed Potatoes & Meatloaf Nuggets
order_id,,,,,,,,,,,,,,,,,,,,,
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#relace NAN with 0
table1=table1.fillna(0)

In [14]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1
basket_sets = table1.applymap(encode_units)

In [15]:
## Running Apriori to get frequent itemset with a minimum suppirt of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [16]:
frequent_itemsets

,support,itemsets
0,0.019476,[Apple Blueberry Fruit Yogurt Smoothie]
1,0.028872,[Apple Peach Oatmeal Stage 2]
2,0.010287,[Apple Plum Kamut Stage 2 Baby Food]
3,0.013527,[Apple Puffs Finger Foods]
4,0.064262,[Apple and Carrot Stage 2 Baby Food]
5,0.025557,[Apples & Blueberries Stage 2 Baby Food]
6,0.027470,[Apples + Strawberries Organic Nibbly Fingers]
7,0.027489,"[Apples, Guavas & Beets Organic Baby Food]"
8,0.043252,"[Apples, Kale & Avocados Organic Baby Food]"
9,0.029213,"[Apples, Pumpkin & Carrots Organic Baby Food]"


In [17]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

In [18]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Stage 1 Apples Sweet Potatoes Pumpkin & Blueb...,(Organic Stage 2 Peaches & Bananas Baby Food),0.084325,0.047363,0.017221,0.204224,4.311899,0.013227,1.197117
1,(Organic Stage 2 Peaches & Bananas Baby Food),(Stage 1 Apples Sweet Potatoes Pumpkin & Blueb...,0.047363,0.084325,0.017221,0.363600,4.311899,0.013227,1.438836
2,"(Spinach Peas & Pear Stage 2 Baby Food, Apple ...",(Baby Food Stage 2 Blueberry Pear & Purple Car...,0.022488,0.126838,0.011045,0.491154,3.872305,0.008193,1.715966
3,"(Spinach Peas & Pear Stage 2 Baby Food, Baby F...",(Apple and Carrot Stage 2 Baby Food),0.044294,0.064262,0.011045,0.249358,3.880346,0.008199,1.246584
4,"(Apple and Carrot Stage 2 Baby Food, Baby Food...",(Spinach Peas & Pear Stage 2 Baby Food),0.026315,0.117706,0.011045,0.419726,3.565884,0.007948,1.520479
5,(Spinach Peas & Pear Stage 2 Baby Food),"(Apple and Carrot Stage 2 Baby Food, Baby Food...",0.117706,0.026315,0.011045,0.093836,3.565884,0.007948,1.074513
6,(Apple and Carrot Stage 2 Baby Food),"(Spinach Peas & Pear Stage 2 Baby Food, Baby F...",0.064262,0.044294,0.011045,0.171875,3.880346,0.008199,1.154060
7,(Baby Food Stage 2 Blueberry Pear & Purple Car...,"(Spinach Peas & Pear Stage 2 Baby Food, Apple ...",0.126838,0.022488,0.011045,0.087080,3.872305,0.008193,1.070753
8,(Pumpkin & Spinach Stage 2 Baby Food),(Squash & Sweet Peas Stage 2),0.033476,0.040031,0.012807,0.382569,9.556810,0.011467,1.554780
9,(Squash & Sweet Peas Stage 2),(Pumpkin & Spinach Stage 2 Baby Food),0.040031,0.033476,0.012807,0.319924,9.556810,0.011467,1.421200


In [19]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Stage 1 Apples Sweet Potatoes Pumpkin & Blueb...,(Organic Stage 2 Peaches & Bananas Baby Food),0.084325,0.047363,0.017221,0.204224,4.311899,0.013227,1.197117
1,(Organic Stage 2 Peaches & Bananas Baby Food),(Stage 1 Apples Sweet Potatoes Pumpkin & Blueb...,0.047363,0.084325,0.017221,0.363600,4.311899,0.013227,1.438836
2,"(Spinach Peas & Pear Stage 2 Baby Food, Apple ...",(Baby Food Stage 2 Blueberry Pear & Purple Car...,0.022488,0.126838,0.011045,0.491154,3.872305,0.008193,1.715966
3,"(Spinach Peas & Pear Stage 2 Baby Food, Baby F...",(Apple and Carrot Stage 2 Baby Food),0.044294,0.064262,0.011045,0.249358,3.880346,0.008199,1.246584
4,"(Apple and Carrot Stage 2 Baby Food, Baby Food...",(Spinach Peas & Pear Stage 2 Baby Food),0.026315,0.117706,0.011045,0.419726,3.565884,0.007948,1.520479
5,(Spinach Peas & Pear Stage 2 Baby Food),"(Apple and Carrot Stage 2 Baby Food, Baby Food...",0.117706,0.026315,0.011045,0.093836,3.565884,0.007948,1.074513
6,(Apple and Carrot Stage 2 Baby Food),"(Spinach Peas & Pear Stage 2 Baby Food, Baby F...",0.064262,0.044294,0.011045,0.171875,3.880346,0.008199,1.154060
7,(Baby Food Stage 2 Blueberry Pear & Purple Car...,"(Spinach Peas & Pear Stage 2 Baby Food, Apple ...",0.126838,0.022488,0.011045,0.087080,3.872305,0.008193,1.070753
8,(Pumpkin & Spinach Stage 2 Baby Food),(Squash & Sweet Peas Stage 2),0.033476,0.040031,0.012807,0.382569,9.556810,0.011467,1.554780
9,(Squash & Sweet Peas Stage 2),(Pumpkin & Spinach Stage 2 Baby Food),0.040031,0.033476,0.012807,0.319924,9.556810,0.011467,1.421200


In [20]:
final_result.to_csv("Department_babiesApriori.csv",sep=',')

In [21]:
test1=pd.read_csv("Department_babiesApriori.csv")

In [22]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [23]:
test1=test1.replace({"u'": ''}, regex=True)

In [24]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [25]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [26]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [27]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Stage 1 Apples Sweet Potatoes Pumpkin & Bluebe...,Organic Stage 2 Peaches & Bananas Baby Food,0.084325,0.047363,0.017221,0.204224,4.311899,0.013227,1.197117
1,Organic Stage 2 Peaches & Bananas Baby Food,Stage 1 Apples Sweet Potatoes Pumpkin & Bluebe...,0.047363,0.084325,0.017221,0.363600,4.311899,0.013227,1.438836
2,"Spinach Peas & Pear Stage 2 Baby Food, Apple a...",Baby Food Stage 2 Blueberry Pear & Purple Carrot,0.022488,0.126838,0.011045,0.491154,3.872305,0.008193,1.715966
3,"Spinach Peas & Pear Stage 2 Baby Food, Baby Fo...",Apple and Carrot Stage 2 Baby Food,0.044294,0.064262,0.011045,0.249358,3.880346,0.008199,1.246584
4,"Apple and Carrot Stage 2 Baby Food, Baby Food ...",Spinach Peas & Pear Stage 2 Baby Food,0.026315,0.117706,0.011045,0.419726,3.565884,0.007948,1.520479
5,Spinach Peas & Pear Stage 2 Baby Food,"Apple and Carrot Stage 2 Baby Food, Baby Food ...",0.117706,0.026315,0.011045,0.093836,3.565884,0.007948,1.074513
6,Apple and Carrot Stage 2 Baby Food,"Spinach Peas & Pear Stage 2 Baby Food, Baby Fo...",0.064262,0.044294,0.011045,0.171875,3.880346,0.008199,1.154060
7,Baby Food Stage 2 Blueberry Pear & Purple Carrot,"Spinach Peas & Pear Stage 2 Baby Food, Apple a...",0.126838,0.022488,0.011045,0.087080,3.872305,0.008193,1.070753
8,Pumpkin & Spinach Stage 2 Baby Food,Squash & Sweet Peas Stage 2,0.033476,0.040031,0.012807,0.382569,9.556810,0.011467,1.554780
9,Squash & Sweet Peas Stage 2,Pumpkin & Spinach Stage 2 Baby Food,0.040031,0.033476,0.012807,0.319924,9.556810,0.011467,1.421200


In [28]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_babiesApriori.csv")

In [29]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Stage 1 Apples Sweet Potatoes Pumpkin & Bluebe...,Organic Stage 2 Peaches & Bananas Baby Food,0.084325,0.047363,0.017221,0.204224,4.311899,0.013227,1.197117
1,Organic Stage 2 Peaches & Bananas Baby Food,Stage 1 Apples Sweet Potatoes Pumpkin & Bluebe...,0.047363,0.084325,0.017221,0.363600,4.311899,0.013227,1.438836
2,"Spinach Peas & Pear Stage 2 Baby Food, Apple a...",Baby Food Stage 2 Blueberry Pear & Purple Carrot,0.022488,0.126838,0.011045,0.491154,3.872305,0.008193,1.715966
3,"Spinach Peas & Pear Stage 2 Baby Food, Baby Fo...",Apple and Carrot Stage 2 Baby Food,0.044294,0.064262,0.011045,0.249358,3.880346,0.008199,1.246584
4,"Apple and Carrot Stage 2 Baby Food, Baby Food ...",Spinach Peas & Pear Stage 2 Baby Food,0.026315,0.117706,0.011045,0.419726,3.565884,0.007948,1.520479
5,Spinach Peas & Pear Stage 2 Baby Food,"Apple and Carrot Stage 2 Baby Food, Baby Food ...",0.117706,0.026315,0.011045,0.093836,3.565884,0.007948,1.074513
6,Apple and Carrot Stage 2 Baby Food,"Spinach Peas & Pear Stage 2 Baby Food, Baby Fo...",0.064262,0.044294,0.011045,0.171875,3.880346,0.008199,1.154060
7,Baby Food Stage 2 Blueberry Pear & Purple Carrot,"Spinach Peas & Pear Stage 2 Baby Food, Apple a...",0.126838,0.022488,0.011045,0.087080,3.872305,0.008193,1.070753
8,Pumpkin & Spinach Stage 2 Baby Food,Squash & Sweet Peas Stage 2,0.033476,0.040031,0.012807,0.382569,9.556810,0.011467,1.554780
9,Squash & Sweet Peas Stage 2,Pumpkin & Spinach Stage 2 Baby Food,0.040031,0.033476,0.012807,0.319924,9.556810,0.011467,1.421200


In [30]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()